<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных-и-исследовательский-анализ" data-toc-modified-id="Обзор-данных-и-исследовательский-анализ-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных и исследовательский анализ</a></span></li><li><span><a href="#Подготовка-данных-к-обучению-модели" data-toc-modified-id="Подготовка-данных-к-обучению-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных к обучению модели</a></span><ul class="toc-item"><li><span><a href="#Предобработка-на-основе-EDA" data-toc-modified-id="Предобработка-на-основе-EDA-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Предобработка на основе EDA</a></span></li><li><span><a href="#Разделение-данных-на-выборки-и-нормализация" data-toc-modified-id="Разделение-данных-на-выборки-и-нормализация-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Разделение данных на выборки и нормализация</a></span></li></ul></li><li><span><a href="#Baseline" data-toc-modified-id="Baseline-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Baseline</a></span></li><li><span><a href="#Улучшение-сети" data-toc-modified-id="Улучшение-сети-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Улучшение сети</a></span></li><li><span><a href="#Итоговый-вывод" data-toc-modified-id="Итоговый-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Итоговый вывод</a></span></li></ul></div>

# Прогнозирование температуры звезды

**Цель работы** — построить нейронную сеть, определяющую температуру на поверхности открытых звёзд.

Значение метрики, выбранной обсерваторией-заказчиком — RMSE — не должно превышать 4 500 К.

Имеются следующие характеристики звёзд:
* Luminosity(L/Lo) — светимость звезды относительно Солнца (светимость Солнца $L_0 = 3.828 ⋅ 10^{26}$ Вт)
* Radius(R/Ro) — радиус звезды относительно радиуса Солнца (радиус Солнца $R_0 = 6.9551 ⋅ 10^8$ м)
* Absolute magnitude(Mv) — абсолютная звёздная величина (характеризует блеск звезды)
* Star type — тип звезды:
  * 0 — Коричневый карлик
  * 1 — Красный карлик
  * 2 — Белый карлик
  * 3 — Звёзды главной последовательности
  * 4 — Сверхгигант
  * 5 — Гипергигант
* Star color — звёздный цвет на основе спектрального анализа
* Temperature(K) — температура на поверхности звезды в Кельвинах, целевой признак

**Ход работы**

Рассмотрим имеющиеся данные, проведём EDA. После него при необходимости предобработаем данные, подготовим их к обработке моделью.

Далее построим baseline нейронной сети, подберём её основные параметры, оценим базовые предсказания.

После чего попробуем улучшить качество модели, подобрав оптимальную регуляризацию, получим финальное значение метрики.

В итоговом выводе сравним результаты рассмотренных ранее моделей.
 
Таким образом работа будет состоять из пяти этапов:
 1. Обзор данных и исследовательский анализ
 2. Подготовка данных к обучению модели
 3. Baseline
 4. Улучшение сети
 5. Итоговый вывод

## Обзор данных и исследовательский анализ

In [2]:
# импорт библиотек и фиксирование параметров
import random
import numpy as np
import pandas as pd
import re
from math import ceil

import pandas_profiling as pp
import plotly.graph_objects as go

import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, PowerTransformer

SEED = 3
random.seed(3)
np.random.seed(3)
torch.manual_seed(3)
torch.use_deterministic_algorithms(True)

In [3]:
# чтение csv-файла и создание датафрейма
stars = pd.read_csv('6_class.csv')

# получение случайных 10 строк, общей информации
# и описательной статистики о датасете
display(
    stars.sample(10, random_state=SEED),
    stars.info(),
    stars.describe(include='all')
    )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Temperature (K)         240 non-null    int64  
 1   Luminosity(L/Lo)        240 non-null    float64
 2   Radius(R/Ro)            240 non-null    float64
 3   Absolute magnitude(Mv)  240 non-null    float64
 4   Star type               240 non-null    int64  
 5   Star color              240 non-null    object 
dtypes: float64(3), int64(2), object(1)
memory usage: 11.4+ KB


,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color
14,2650,0.000600,0.1400,11.782,1,Red
133,2989,0.008700,0.3400,13.120,1,Red
189,3523,0.000957,0.1290,16.350,0,Red
6,2637,0.000730,0.1270,17.220,0,Red
61,3432,0.000670,0.1900,16.940,0,Red
132,3100,0.008000,0.3100,11.170,1,Red
197,3496,0.001250,0.3360,14.940,1,Red
154,25070,14500.000000,5.9200,-3.980,3,Blue-white
43,3200,195000.000000,17.0000,-7.220,4,Red
206,24020,0.001590,0.0127,10.550,2,Blue


None

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color
count,240.000000,240.000000,240.000000,240.000000,240.000000,240
unique,NaN,NaN,NaN,NaN,NaN,19
top,NaN,NaN,NaN,NaN,NaN,Red
freq,NaN,NaN,NaN,NaN,NaN,112
mean,10497.462500,107188.361635,237.157781,4.382396,2.500000,NaN
std,9552.425037,179432.244940,517.155763,10.532512,1.711394,NaN
min,1939.000000,0.000080,0.008400,-11.920000,0.000000,NaN
25%,3344.250000,0.000865,0.102750,-6.232500,1.000000,NaN
50%,5776.000000,0.070500,0.762500,8.313000,2.500000,NaN
75%,15055.500000,198050.000000,42.750000,13.697500,4.000000,NaN


Детальнее изучим данные с помощью EDA.

In [4]:
#проведение EDA
pp.ProfileReport(stars, vars={'num': {'low_categorical_threshold': 7}})

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

**Промежуточные выводы**

Можно наблюдать, что числовые признаки в основном распределены сильно неравномерно, имеется достаточное количество выбросов, однако все они не противоречат фактам и природе данных, избавляться от них не будем, ограничимся масштабированием; большинство звёзд достаточно холодные, относительные величины по большей части идентичны показателям Солнца.

Видно, что датасет собирался равномерно по типу звёзд, у всех 6 типов одинаковое количество представителей.

Среди цветов звёзд точно имеются неявные дубликаты, что стоит исправить, сведя количество категорий к разумному минимуму.

Наблюдается достаточно низкая корреляция относительного радиуса с целевым признаком по различным типам корреляций, возможно, стоит попробовать построить сеть в том числе и без использования данного признака.

Также изменим названия колонок датасета в соответствии с `snake_case` для дальнейшего удобства.

## Подготовка данных к обучению модели

### Предобработка на основе EDA

Исправим ранее обнаруженные артефакты.

In [5]:
# переименование столбцов
stars.columns = stars.columns.str.replace(
    r'\s*\(.*?\)',
    '',
    regex=True
    ).str.replace(
        ' ',
        '_').str.lower()

# проверка изменений
stars.columns

Index(['temperature', 'luminosity', 'radius', 'absolute_magnitude',
       'star_type', 'star_color'],
      dtype='object')

In [6]:
# просмотр всех значений цветов звёзд
stars.star_color.value_counts()

Red                   112
Blue                   55
Blue-white             26
Blue White             10
yellow-white            8
White                   7
Blue white              3
Yellowish White         3
white                   3
Orange                  2
Whitish                 2
yellowish               2
Yellowish               1
Blue white              1
Pale yellow orange      1
White-Yellow            1
Orange-Red              1
Blue                    1
Blue-White              1
Name: star_color, dtype: int64

In [7]:
# сокращение категорий и избавление от неявных дубликатов
stars.star_color = stars.star_color.str.strip().str.lower().replace({
    r'.*orange.*': 'yellow',
    'yellowish': 'yellow',
    'whitish': 'white',
    ' ': '-',
    'white-yellow': 'yellow-white'
    }, regex=True)

# просмотр изменений
stars.star_color.value_counts()

red             112
blue             56
blue-white       41
yellow-white     12
white            12
yellow            7
Name: star_color, dtype: int64

### Разделение данных на выборки и нормализация

Сформируем обучающую, валидационную и тестовую выборки, закодируем категориальные и масштабируем количественные признаки. Сформируем также выборки без информации о радиусе звёзд для большей вариативности обучения.

In [8]:
# разделение данных на выборки
features = stars.drop('temperature', axis=1)
target = stars.temperature

train_features, second_features, train_target, second_target = train_test_split(
    features,
    target,
    test_size=.4,
    random_state=SEED,
    shuffle=True
    )
valid_features, test_features, valid_target, test_target = train_test_split(
    second_features,
    second_target,
    test_size=.5,
    random_state=SEED,
    shuffle=True
    )

train_features = train_features.reset_index(drop=True)
valid_features = valid_features.reset_index(drop=True)
test_features = test_features.reset_index(drop=True)
train_target = train_target.reset_index(drop=True)
valid_target = valid_target.reset_index(drop=True)
test_target = test_target.reset_index(drop=True)

# проверка разбиения
print(
    train_features.shape,
    valid_features.shape,
    test_features.shape,
    '\n',
    train_target.shape,
    valid_target.shape,
    test_target.shape
    )

(144, 5) (48, 5) (48, 5) 
 (144,) (48,) (48,)


In [9]:
# кодирование категориальных признаков
categorical = ['star_type', 'star_color']
cols = [
    '1',
    '2',
    '3',
    '4',
    '5',
    'blue-white',
    'red',
    'white',
    'yellow',
    'yellow-white'
    ]
ohe = OneHotEncoder(drop='first', dtype='int')
ohe.fit(train_features[categorical])

enc_train_features = pd.DataFrame(
    ohe.transform(train_features[categorical]).toarray(),
    columns=cols
    )
train_features = train_features.join(enc_train_features)
train_features = train_features.drop(categorical, axis=1)

enc_valid_features = pd.DataFrame(
    ohe.transform(valid_features[categorical]).toarray(),
    columns=cols
    )
valid_features = valid_features.join(enc_valid_features)
valid_features = valid_features.drop(categorical, axis=1)

enc_test_features = pd.DataFrame(
    ohe.transform(test_features[categorical]).toarray(),
    columns=cols
    )
test_features = test_features.join(enc_test_features)
test_features = test_features.drop(categorical, axis=1)

# проверка результата
display(
    train_features.sample(5, random_state=SEED),
    valid_features.sample(5, random_state=SEED),
    test_features.sample(5, random_state=SEED)
    )

,luminosity,radius,absolute_magnitude,1,2,3,4,5,blue-white,red,white,yellow,yellow-white
25,382993.000000,1494.0000,-8.84,0,0,0,0,1,1,0,0,0,0
6,0.000896,0.0782,19.56,0,0,0,0,0,0,1,0,0,0
3,0.022000,0.3800,10.12,1,0,0,0,0,0,1,0,0,0
45,0.003400,0.2400,13.46,1,0,0,0,0,0,1,0,0,0
40,0.000430,0.0912,17.16,0,0,0,0,0,0,1,0,0,0


,luminosity,radius,absolute_magnitude,1,2,3,4,5,blue-white,red,white,yellow,yellow-white
12,0.000138,0.1030,20.060,0,0,0,0,0,0,1,0,0,0
38,0.000452,0.0987,17.340,0,0,0,0,0,0,1,0,0,0
9,0.021000,0.2730,12.300,1,0,0,0,0,0,1,0,0,0
45,6748.000000,6.6400,-2.550,0,0,1,0,0,1,0,0,0,0
31,552.000000,5.8560,0.013,0,0,1,0,0,1,0,0,0,0


,luminosity,radius,absolute_magnitude,1,2,3,4,5,blue-white,red,white,yellow,yellow-white
12,1278.000000,5.680,-3.32,0,0,1,0,0,1,0,0,0,0
38,0.000200,0.160,16.65,0,0,0,0,0,0,1,0,0,0
9,0.000849,0.112,19.45,0,0,0,0,0,0,1,0,0,0
45,0.000400,0.196,13.21,1,0,0,0,0,0,1,0,0,0
31,235000.000000,83.000,-6.89,0,0,0,1,0,0,0,0,0,0


In [10]:
# масштабирование количественных признаков
numerical = ['luminosity', 'radius', 'absolute_magnitude']
transformer = PowerTransformer()

train_features[numerical] = transformer.fit_transform(train_features[numerical])
valid_features[numerical] = transformer.transform(valid_features[numerical])
test_features[numerical] = transformer.transform(test_features[numerical])

# формирование выборок без сомнительного, возможно неинформативного признака
abb_train_features = train_features.drop('radius', axis=1)
abb_valid_features = valid_features.drop('radius', axis=1)
abb_test_features = test_features.drop('radius', axis=1)

# проверка результата
display(
    train_features.sample(5, random_state=SEED),
    abb_train_features.sample(5, random_state=SEED),
    valid_features.sample(5, random_state=SEED),
    abb_valid_features.sample(5, random_state=SEED),
    test_features.sample(5, random_state=SEED),
    abb_test_features.sample(5, random_state=SEED),
    )

,luminosity,radius,absolute_magnitude,1,2,3,4,5,blue-white,red,white,yellow,yellow-white
25,1.231982,1.547635,-1.250619,0,0,0,0,1,1,0,0,0,0
6,-0.929713,-0.947260,1.409431,0,0,0,0,0,0,1,0,0,0
3,-0.924508,-0.753378,0.645500,1,0,0,0,0,0,1,0,0,0
45,-0.929089,-0.835749,0.920071,1,0,0,0,0,0,1,0,0,0
40,-0.929829,-0.937527,1.218478,0,0,0,0,0,0,1,0,0,0


,luminosity,absolute_magnitude,1,2,3,4,5,blue-white,red,white,yellow,yellow-white
25,1.231982,-1.250619,0,0,0,0,1,1,0,0,0,0
6,-0.929713,1.409431,0,0,0,0,0,0,1,0,0,0
3,-0.924508,0.645500,1,0,0,0,0,0,1,0,0,0
45,-0.929089,0.920071,1,0,0,0,0,0,1,0,0,0
40,-0.929829,1.218478,0,0,0,0,0,0,1,0,0,0


,luminosity,radius,absolute_magnitude,1,2,3,4,5,blue-white,red,white,yellow,yellow-white
12,-0.929902,-0.928820,1.448987,0,0,0,0,0,0,1,0,0,0
38,-0.929823,-0.931979,1.232865,0,0,0,0,0,0,1,0,0,0
9,-0.924752,-0.815292,0.825341,1,0,0,0,0,0,1,0,0,0
45,0.737952,0.274158,-0.519992,0,0,1,0,0,1,0,0,0,0
31,0.358783,0.222904,-0.246880,0,0,1,0,0,1,0,0,0,0


,luminosity,absolute_magnitude,1,2,3,4,5,blue-white,red,white,yellow,yellow-white
12,-0.929902,1.448987,0,0,0,0,0,0,1,0,0,0
38,-0.929823,1.232865,0,0,0,0,0,0,1,0,0,0
9,-0.924752,0.825341,1,0,0,0,0,0,1,0,0,0
45,0.737952,-0.519992,0,0,1,0,0,1,0,0,0,0
31,0.358783,-0.246880,0,0,1,0,0,1,0,0,0,0


,luminosity,radius,absolute_magnitude,1,2,3,4,5,blue-white,red,white,yellow,yellow-white
12,0.492905,0.210357,-0.606019,0,0,1,0,0,1,0,0,0,0
38,-0.929886,-0.888385,1.177653,0,0,0,0,0,0,1,0,0,0
9,-0.929724,-0.922259,1.400719,0,0,0,0,0,0,1,0,0,0
45,-0.929836,-0.864139,0.899707,1,0,0,0,0,0,1,0,0,0
31,1.179003,1.084306,-1.018448,0,0,0,1,0,0,0,0,0,0


,luminosity,absolute_magnitude,1,2,3,4,5,blue-white,red,white,yellow,yellow-white
12,0.492905,-0.606019,0,0,1,0,0,1,0,0,0,0
38,-0.929886,1.177653,0,0,0,0,0,0,1,0,0,0
9,-0.929724,1.400719,0,0,0,0,0,0,1,0,0,0
45,-0.929836,0.899707,1,0,0,0,0,0,1,0,0,0
31,1.179003,-1.018448,0,0,0,1,0,0,0,0,0,0


**Промежуточные итоги**

Количественные и категориальные данные были обработаны с помощью масштабирования и горячего кодирования соответственно. В категориальных признаках перед кодированием были объединены некоторые задублированные, а также небольшие категории для итогового уменьшения размерности данных и более корректного их разделения на выборки. Была определена и отдельная выборка без сомнительного признака.

## Baseline

Создадим простую модель нейронной сети, подберём количество нейронов, слоёв и функцию активации.

In [11]:
# формирование тензоров выборок
f_train = torch.FloatTensor(train_features.values)
f_valid = torch.FloatTensor(valid_features.values)
f_test = torch.FloatTensor(test_features.values)

abb_f_train = torch.FloatTensor(abb_train_features.values)
abb_f_valid = torch.FloatTensor(abb_valid_features.values)
abb_f_test = torch.FloatTensor(abb_test_features.values)

t_train = torch.FloatTensor(train_target.values)
t_valid = torch.FloatTensor(valid_target.values)
t_test = torch.FloatTensor(test_target.values)

In [12]:
# определение функции для построения графиков
def fit_plot(loss, metric, abb=0):
    fig = go.Figure()
    if abb == 1:
        fig.add_trace(go.Scatter(
            x=np.array(range(len(loss)))*10,
            y=loss,
            name='Тренировка',
            line=dict(color='darkblue', width=3)
            ))
        fig.add_trace(go.Scatter(
            x=np.array(range(len(metric)))*10,
            y=metric,
            name='Валидация',
            line=dict(color='crimson', width=3)
            ))
        fig.update_layout(
            width=1000,
            height=500,
            legend_orientation='h',
            title=dict(
                text='Значения функции потерь при тренировке и метрики на валидации без одного признака',
                x=.5
                ),
            xaxis_title='Эпоха',
            yaxis_title='RMSE'
            )
    else:
        fig.add_trace(go.Scatter(
            x=np.array(range(len(loss)))*10,
            y=loss,
            name='Тренировка',
            line=dict(color='darkcyan', width=3)
            ))
        fig.add_trace(go.Scatter(
            x=np.array(range(len(metric)))*10,
            y=metric,
            name='Валидация',
            line=dict(color='coral', width=3)
            ))
        fig.update_layout(
            width=1000,
            height=500,
            legend_orientation='h',
            title=dict(
                text='Значения функции потерь при тренировке и метрики на валидации со всеми признаками',
                x=.5
                ),
            xaxis_title='Эпоха',
            yaxis_title='RMSE'
            )
    fig.show()

Сначала попробуем разное количество нейронов.

In [13]:
# определение класса нейронной сети
class Net(nn.Module):
    def __init__(
        self,
        n_in_neurons,
        n_hidden_neurons_1,
        n_hidden_neurons_2,
        n_out_neurons
        ):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
        self.act1 = nn.ReLU()

        self.fc2 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2)
        self.act2 =  nn.ReLU()

        self.fc3 = nn.Linear(n_hidden_neurons_2, n_out_neurons)

        nn.init.kaiming_normal_(self.fc1.weight, mode='fan_in')
        nn.init.normal_(self.fc1.bias, mean=.5, std=.7)
        
        nn.init.kaiming_normal_(self.fc2.weight, mode='fan_in')
        nn.init.normal_(self.fc2.bias, mean=.5, std=.7)

        nn.init.kaiming_normal_(self.fc3.weight, mode='fan_in')
        nn.init.normal_(self.fc3.bias, mean=.5, std=.7)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)

        x = self.fc2(x)
        x = self.act2(x)

        x = self.fc3(x)
        return x

In [14]:
# определение экземпляров сетей и количества нейронов
n_in_neurons = 13
n_abb_in_neurons = 12
n_hidden_neurons_1_m = 24
n_hidden_neurons_2_m = 12
n_hidden_neurons_1_l = 18
n_hidden_neurons_2_l = 8
n_out_neurons = 1 

net_m = Net(
    n_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons
    )
abb_net_m = Net(
    n_abb_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons
    )
net_l = Net(
    n_in_neurons,
    n_hidden_neurons_1_l,
    n_hidden_neurons_2_l,
    n_out_neurons
    )
abb_net_l = Net(
    n_abb_in_neurons,
    n_hidden_neurons_1_l,
    n_hidden_neurons_2_l,
    n_out_neurons
    )
# определение оптимизаторов, функции потерь и количества эпох
optimizer_m = torch.optim.Adamax(net_m.parameters(), lr=5e-2)
abb_optimizer_m = torch.optim.Adamax(abb_net_m.parameters(), lr=5e-2)

optimizer_l = torch.optim.Adamax(net_l.parameters(), lr=5e-2)
abb_optimizer_l = torch.optim.Adamax(abb_net_l.parameters(), lr=5e-2)

loss = nn.MSELoss()

num_epochs = 10000

# создание списков для дальнейшего отображения графиков
train_loss_m = []
valid_metric_m = []
abb_train_loss_m = []
abb_valid_metric_m = []

train_loss_l = []
valid_metric_l = []
abb_train_loss_l = []
abb_valid_metric_l = []

In [15]:
# определение функции для обучения нейронной сети
def fit(
    net,
    optimizer,
    train_loss,
    valid_metric,
    patience=1,
    max_delta=0,
    abb=0
    ):
    counter = 0
    min_valid_metric = np.inf
    
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        
        if abb == 1:
            preds = net.forward(abb_f_train).flatten()
        else:
            preds = net.forward(f_train).flatten()
        loss_value = loss(preds, t_train)
        loss_value.backward()
    
        optimizer.step()

        if epoch % 10 == 0 or epoch == num_epochs - 1:
            train_loss.append(torch.sqrt(loss_value).detach())

            net.eval()
            if abb==1:
                valid_preds = net.forward(abb_f_valid).flatten()
            else:
                valid_preds = net.forward(f_valid).flatten()
            rmse = torch.sqrt(loss(
                valid_preds,
                t_valid
                )).detach()
            valid_metric.append(rmse)
            
            if rmse < min_valid_metric:
                min_valid_metric = rmse
                counter = 0
            elif rmse >= min_valid_metric:
                counter += 1
            if counter >= patience or rmse - min_valid_metric >= max_delta:
                break

In [16]:
# обучение нейронных сетей
fit(
    net_m,
    optimizer_m,
    train_loss_m,
    valid_metric_m,
    10,
    50
    )
fit(
    abb_net_m,
    abb_optimizer_m,
    abb_train_loss_m,
    abb_valid_metric_m,
    10,
    50,
    1
    )
fit(
    net_l,
    optimizer_l,
    train_loss_l,
    valid_metric_l,
    10,
    50
    )
fit(
    abb_net_l,
    abb_optimizer_l,
    abb_train_loss_l,
    abb_valid_metric_l,
    10,
    50,
    1
    )

In [17]:
# просмотр процесса обучения сетей с большим количеством нейронов
fit_plot(train_loss_m, valid_metric_m)
fit_plot(abb_train_loss_m, abb_valid_metric_m, 1)

In [18]:
# просмотр процесса обучения сетей с меньшим количеством нейронов
fit_plot(train_loss_l, valid_metric_l)
fit_plot(abb_train_loss_l, abb_valid_metric_l, 1)

In [19]:
# сравнение наилучших метрик сетей на валидации
print(
    'Наименьшая RMSE сети с большим количеством нейронов и всеми признаками —',
    np.min(valid_metric_m),
    f'— на {np.argmin(valid_metric_m)*10} эпохе'
    )
print(
    'Наименьшая RMSE сети с большим количеством нейронов и без одного признака —',
    np.min(abb_valid_metric_m),
    f'— на {np.argmin(abb_valid_metric_m)*10} эпохе'
    )
print(
    'Наименьшая RMSE сети с меньшим количеством нейронов и всеми признаками —',
    np.min(valid_metric_l),
    f'— на {np.argmin(valid_metric_l)*10} эпохе'
    )
print(
    'Наименьшая RMSE сети с меньшим количеством нейронов и без одного признака —',
    np.min(abb_valid_metric_l),
    f'— на {np.argmin(abb_valid_metric_l)*10} эпохе'
    )

Наименьшая RMSE сети с большим количеством нейронов и всеми признаками — 4622.0547 — на 900 эпохе
Наименьшая RMSE сети с большим количеством нейронов и без одного признака — 4729.5415 — на 1340 эпохе
Наименьшая RMSE сети с меньшим количеством нейронов и всеми признаками — 14826.696 — на 10000 эпохе
Наименьшая RMSE сети с меньшим количеством нейронов и без одного признака — 4664.612 — на 3230 эпохе


Лучший результат у сети с большим количеством нейронов. Сравним с сетью, имеющей больше слоёв. 

In [20]:
# определение класса нейронной сети с большим количеством слоёв
class NetExt(nn.Module):
    def __init__(
        self,
        n_in_neurons,
        n_hidden_neurons_1,
        n_hidden_neurons_2,
        n_hidden_neurons_3,
        n_out_neurons
        ):
        super(NetExt, self).__init__()

        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
        self.act1 = nn.ReLU()

        self.fc2 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2)
        self.act2 =  nn.ReLU()

        self.fc3 = nn.Linear(n_hidden_neurons_2, n_hidden_neurons_3)
        self.act3 =  nn.ReLU()

        self.fc4 = nn.Linear(n_hidden_neurons_3, n_out_neurons)

        nn.init.kaiming_normal_(self.fc1.weight, mode='fan_in')
        nn.init.normal_(self.fc1.bias, mean=.5, std=.7)
        
        nn.init.kaiming_normal_(self.fc2.weight, mode='fan_in')
        nn.init.normal_(self.fc2.bias, mean=.5, std=.7)

        nn.init.kaiming_normal_(self.fc3.weight, mode='fan_in')
        nn.init.normal_(self.fc3.bias, mean=.5, std=.7)

        nn.init.kaiming_normal_(self.fc4.weight, mode='fan_in')
        nn.init.normal_(self.fc4.bias, mean=.5, std=.7)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)

        x = self.fc2(x)
        x = self.act2(x)

        x = self.fc3(x)
        x = self.act3(x)

        x = self.fc4(x)
        return x

In [21]:
# определение экземпляров сетей и количества нейронов
n_hidden_neurons_3 = 10

net_ext = NetExt(
    n_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_hidden_neurons_3,
    n_out_neurons
    )
abb_net_ext = NetExt(
    n_abb_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_hidden_neurons_3,
    n_out_neurons
    )
# определение оптимизаторов
optimizer_ext = torch.optim.Adamax(net_ext.parameters(), lr=5e-2)
abb_optimizer_ext = torch.optim.Adamax(abb_net_ext.parameters(), lr=5e-2)

# создание списков для дальнейшего отображения графиков
train_loss_ext = []
valid_metric_ext = []
abb_train_loss_ext = []
abb_valid_metric_ext = []

In [22]:
# обучение нейронных сетей
fit(
    net_ext,
    optimizer_ext,
    train_loss_ext,
    valid_metric_ext,
    10,
    50
    )
fit(
    abb_net_ext,
    abb_optimizer_ext,
    abb_train_loss_ext,
    abb_valid_metric_ext,
    10,
    50,
    1
    )

In [23]:
# просмотр процесса обучения сетей с меньшим количеством слоёв
fit_plot(train_loss_m, valid_metric_m)
fit_plot(abb_train_loss_m, abb_valid_metric_m, 1)

In [24]:
# просмотр процесса обучения сетей с большим количеством слоёв
fit_plot(train_loss_ext, valid_metric_ext)
fit_plot(abb_train_loss_ext, abb_valid_metric_ext, 1)

In [25]:
# сравнение наилучших метрик сетей на валидации
print(
    'Наименьшая RMSE сети с меньшим количеством слоёв и всеми признаками —',
    np.min(valid_metric_m),
    f'— на {np.argmin(valid_metric_m)*10} эпохе'
    )
print(
    'Наименьшая RMSE сети с меньшим количеством слоёв и без одного признака —',
    np.min(abb_valid_metric_m),
    f'— на {np.argmin(abb_valid_metric_m)*10} эпохе'
    )
print(
    'Наименьшая RMSE сети с большим количеством слоёв и всеми признаками —',
    np.min(valid_metric_ext),
    f'— на {np.argmin(valid_metric_ext)*10} эпохе'
    )
print(
    'Наименьшая RMSE сети с большим количеством слоёв и без одного признака —',
    np.min(abb_valid_metric_ext),
    f'— на {np.argmin(abb_valid_metric_ext)*10} эпохе'
    )

Наименьшая RMSE сети с меньшим количеством слоёв и всеми признаками — 4622.0547 — на 900 эпохе
Наименьшая RMSE сети с меньшим количеством слоёв и без одного признака — 4729.5415 — на 1340 эпохе
Наименьшая RMSE сети с большим количеством слоёв и всеми признаками — 4682.468 — на 690 эпохе
Наименьшая RMSE сети с большим количеством слоёв и без одного признака — 4761.989 — на 410 эпохе


Качество лучше у сети с меньшим количеством слоёв. Далее попробуем разные функции активации.

In [26]:
# определение класса нейронной сети с функцией активации ELU
class NetELU(nn.Module):
    def __init__(
        self,
        n_in_neurons,
        n_hidden_neurons_1,
        n_hidden_neurons_2,
        n_out_neurons
        ):
        super(NetELU, self).__init__()

        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
        self.act1 = nn.ELU(.001)

        self.fc2 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2)
        self.act2 =  nn.ELU(.001)

        self.fc3 = nn.Linear(n_hidden_neurons_2, n_out_neurons)

        nn.init.kaiming_normal_(self.fc1.weight, mode='fan_in')
        nn.init.normal_(self.fc1.bias, mean=.5, std=.7)
        
        nn.init.kaiming_normal_(self.fc2.weight, mode='fan_in')
        nn.init.normal_(self.fc2.bias, mean=.5, std=.7)

        nn.init.kaiming_normal_(self.fc3.weight, mode='fan_in')
        nn.init.normal_(self.fc3.bias, mean=.5, std=.7)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)

        x = self.fc2(x)
        x = self.act2(x)

        x = self.fc3(x)
        return x

In [27]:
# определение экземпляров сетей
net_elu = NetELU(
    n_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons
    )
abb_net_elu = NetELU(
    n_abb_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons
    )
# определение оптимизаторов
optimizer_elu = torch.optim.Adamax(net_elu.parameters(), lr=5e-2)
abb_optimizer_elu = torch.optim.Adamax(abb_net_elu.parameters(), lr=5e-2)

# создание списков для дальнейшего отображения графиков
train_loss_elu = []
valid_metric_elu = []
abb_train_loss_elu = []
abb_valid_metric_elu = []

In [28]:
# обучение нейронных сетей
fit(
    net_elu,
    optimizer_elu,
    train_loss_elu,
    valid_metric_elu,
    10,
    50
    )
fit(
    abb_net_elu,
    abb_optimizer_elu,
    abb_train_loss_elu,
    abb_valid_metric_elu,
    10,
    50,
    1
    )

In [29]:
# просмотр процесса обучения сетей с ReLU
fit_plot(train_loss_m, valid_metric_m)
fit_plot(abb_train_loss_m, abb_valid_metric_m, 1)

In [30]:
# просмотр процесса обучения сетей с ELU
fit_plot(train_loss_elu, valid_metric_elu)
fit_plot(abb_train_loss_elu, abb_valid_metric_elu, 1)

In [31]:
# сравнение наилучших метрик сетей на валидации
print(
    'Наименьшая RMSE сети с ReLU и всеми признаками —',
    np.min(valid_metric_m),
    f'— на {np.argmin(valid_metric_m)*10} эпохе'
    )
print(
    'Наименьшая RMSE сети с ReLU и без одного признака —',
    np.min(abb_valid_metric_m),
    f'— на {np.argmin(abb_valid_metric_m)*10} эпохе'
    )
print(
    'Наименьшая RMSE сети с ELU и всеми признаками —',
    np.min(valid_metric_elu),
    f'— на {np.argmin(valid_metric_elu)*10} эпохе'
    )
print(
    'Наименьшая RMSE сети с ELU и без одного признака —',
    np.min(abb_valid_metric_elu),
    f'— на {np.argmin(abb_valid_metric_elu)*10} эпохе'
    )

Наименьшая RMSE сети с ReLU и всеми признаками — 4622.0547 — на 900 эпохе
Наименьшая RMSE сети с ReLU и без одного признака — 4729.5415 — на 1340 эпохе
Наименьшая RMSE сети с ELU и всеми признаками — 4558.878 — на 1060 эпохе
Наименьшая RMSE сети с ELU и без одного признака — 4579.826 — на 1860 эпохе


Посмотрим на график "Факт — Прогноз", наглядно показывающий разницу между действительными и предсказанными наилучшей на данный момент конфигурацией нейросети температурами звёзд.

In [32]:
# получение предсказаний
valid_predictions = net_elu.forward(f_valid).flatten().detach()

# построение графика
valid_fig = go.Figure()
valid_fig.add_trace(go.Bar(
    x=np.array(range(len(t_valid))),
    y=t_valid,
    name='Факт',
    width=.9,
    marker_color='dodgerblue'
    ))
valid_fig.add_trace(go.Bar(
    x=np.array(range(len(valid_predictions))),
    y=valid_predictions,
    name='Прогноз',
    width=.3,
    marker_color='orange'
    ))
valid_fig.update_layout(
    barmode='overlay',
    width=1000,
    height=500,
    legend_orientation='h',
    title=dict(
        text='Истинные и предсказанные температуры звёзд на валидации',
        x=.5
        ),
    xaxis_title='Номер звезды в выборке',
    yaxis_title='Температура звезды, К'
    )
valid_fig.show()

**Промежуточные выводы**

По результатам рассмотрения разных архитектур нейросетей наилучшей комбинацией модели является сеть со всего двумя скрытыми слоями, достаточно большим количеством нейронов относительно числа признаков на входном слое и функцией активации ELU.

Наличие или отсутствие такого признака звезды, как радиус, даёт различный результат в зависимости от конкретной архитектуры сети — иногда качество растёт, а иногда падает. В таком случае стоит и при дальнейших экспериментах с моделью рассматривать оба различных набора признаков.

Сети достигают минимальной RMSE достатчно рано — чаще всего в пределах 2 000 эпох.

Значение метрики на валидационной выборке уже на данном этапе практически достигло требуемого порога. Попробуем ещё более улучшить качество нейросети.

## Улучшение сети

Рассмотрим ещё некоторые подходы в рамках обучения сети для возможного повышения её качества. Попробуем подавать данные для обучения не все сразу целиком, а батчами.
Также протестируем технику регуляризации Dropout. Подберём как наилучший размер батча, так и долю неработающих нейронов.

In [33]:
# определение класса нейронной сети с Dropout
class NetDp(nn.Module):
    def __init__(
        self,
        n_in_neurons,
        n_hidden_neurons_1,
        n_hidden_neurons_2,
        n_out_neurons,
        part
        ):
        super(NetDp, self).__init__()

        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
        self.act1 = nn.ELU(.001)
        self.dp1 = nn.Dropout(p=part)

        self.fc2 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2)
        self.act2 =  nn.ELU(.001)
        self.dp2 = nn.Dropout(p=part)

        self.fc3 = nn.Linear(n_hidden_neurons_2, n_out_neurons)

        nn.init.kaiming_normal_(self.fc1.weight, mode='fan_in')
        nn.init.normal_(self.fc1.bias, mean=.5, std=.7)
        
        nn.init.kaiming_normal_(self.fc2.weight, mode='fan_in')
        nn.init.normal_(self.fc2.bias, mean=.5, std=.7)

        nn.init.kaiming_normal_(self.fc3.weight, mode='fan_in')
        nn.init.normal_(self.fc3.bias, mean=.5, std=.7)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.dp1(x)

        x = self.fc2(x)
        x = self.act2(x)
        x = self.dp2(x)

        x = self.fc3(x)
        return x

In [34]:
# определение экземпляров сетей
dp_net_2 = NetDp(
    n_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons,
    .2
    )
abb_dp_net_2 = NetDp(
    n_abb_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons,
    .2
    )
dp_net_4 = NetDp(
    n_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons,
    .4
    )
abb_dp_net_4 = NetDp(
    n_abb_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons,
    .4
    )
dp_net_6 = NetDp(
    n_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons,
    .6
    )
abb_dp_net_6 = NetDp(
    n_abb_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons,
    .6
    )
dp_net_8 = NetDp(
    n_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons,
    .8
    )
abb_dp_net_8 = NetDp(
    n_abb_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons,
    .8
    )
# определение оптимизаторов
dp_optimizer_2 = torch.optim.Adamax(dp_net_2.parameters(), lr=5e-2)
abb_dp_optimizer_2 = torch.optim.Adamax(abb_dp_net_2.parameters(), lr=5e-2)
dp_optimizer_4 = torch.optim.Adamax(dp_net_4.parameters(), lr=5e-2)
abb_dp_optimizer_4 = torch.optim.Adamax(abb_dp_net_4.parameters(), lr=5e-2)
dp_optimizer_6 = torch.optim.Adamax(dp_net_6.parameters(), lr=5e-2)
abb_dp_optimizer_6 = torch.optim.Adamax(abb_dp_net_6.parameters(), lr=5e-2)
dp_optimizer_8 = torch.optim.Adamax(dp_net_8.parameters(), lr=5e-2)
abb_dp_optimizer_8 = torch.optim.Adamax(abb_dp_net_8.parameters(), lr=5e-2)

# создание списков cо значениями метрик
dp_train_loss_2 = []
dp_valid_metric_2 = []
abb_dp_train_loss_2 = []
abb_dp_valid_metric_2 = []
dp_train_loss_4 = []
dp_valid_metric_4 = []
abb_dp_train_loss_4 = []
abb_dp_valid_metric_4 = []
dp_train_loss_6 = []
dp_valid_metric_6 = []
abb_dp_train_loss_6 = []
abb_dp_valid_metric_6 = []
dp_train_loss_8 = []
dp_valid_metric_8 = []
abb_dp_train_loss_8 = []
abb_dp_valid_metric_8 = []

При использовании Dropout поднимем пороговые значения остановки обучения, так как обычно в таком случае сети требуется больше эпох для её сходимости.

In [35]:
# обучение нейронных сетей
fit(
    dp_net_2,
    dp_optimizer_2,
    dp_train_loss_2,
    dp_valid_metric_2,
    20,
    100
    )
fit(
    abb_dp_net_2,
    abb_dp_optimizer_2,
    abb_dp_train_loss_2,
    abb_dp_valid_metric_2,
    20,
    100,
    1
    )
fit(
    dp_net_4,
    dp_optimizer_4,
    dp_train_loss_4,
    dp_valid_metric_4,
    20,
    100
    )
fit(
    abb_dp_net_4,
    abb_dp_optimizer_4,
    abb_dp_train_loss_4,
    abb_dp_valid_metric_4,
    20,
    100,
    1
    )
fit(
    dp_net_6,
    dp_optimizer_6,
    dp_train_loss_6,
    dp_valid_metric_6,
    20,
    100
    )
fit(
    abb_dp_net_6,
    abb_dp_optimizer_6,
    abb_dp_train_loss_6,
    abb_dp_valid_metric_6,
    20,
    100,
    1
    )
fit(
    dp_net_8,
    dp_optimizer_8,
    dp_train_loss_8,
    dp_valid_metric_8,
    20,
    100
    )
fit(
    abb_dp_net_8,
    abb_dp_optimizer_8,
    abb_dp_train_loss_8,
    abb_dp_valid_metric_8,
    20,
    100,
    1
    )

In [36]:
# создание сравнительной таблицы показателей метрики
rmses = pd.DataFrame(
    index=['baseline'],
    columns=['all_features', 'without_one_feature']
    )
rmses.loc['baseline', 'all_features'] = np.min(valid_metric_elu)
rmses.loc['baseline', 'without_one_feature'] = np.min(abb_valid_metric_elu)

# сравнение наилучших метрик сетей на валидации
print(
    'Наименьшая RMSE сети без Dropout и со всеми признаками —',
    np.min(valid_metric_m),
    f'— на {np.argmin(valid_metric_m)*10} эпохе'
    )
print(
    'Наименьшая RMSE сети без Dropout и одного признака —',
    np.min(abb_valid_metric_m),
    f'— на {np.argmin(abb_valid_metric_m)*10} эпохе',
    '\n'
    )
print(
    'Наименьшая RMSE сети с долей Dropout 0.2 и всеми признаками —',
    np.min(dp_valid_metric_2),
    f'— на {np.argmin(dp_valid_metric_2)*10} эпохе'
    )
rmses.loc['dropout_0.2', 'all_features'] = np.min(dp_valid_metric_2)
print(
    'Наименьшая RMSE сети с долей Dropout 0.2 и без одного признака —',
    np.min(abb_dp_valid_metric_2),
    f'— на {np.argmin(abb_dp_valid_metric_2)*10} эпохе',
    '\n'
    )
rmses.loc['dropout_0.2', 'without_one_feature'] = np.min(abb_dp_valid_metric_2)
print(
    'Наименьшая RMSE сети с долей Dropout 0.4 и всеми признаками —',
    np.min(dp_valid_metric_4),
    f'— на {np.argmin(dp_valid_metric_4)*10} эпохе'
    )
rmses.loc['dropout_0.4', 'all_features'] = np.min(dp_valid_metric_4)
print(
    'Наименьшая RMSE сети с долей Dropout 0.4 и без одного признака —',
    np.min(abb_dp_valid_metric_4),
    f'— на {np.argmin(abb_dp_valid_metric_4)*10} эпохе',
    '\n'
    )
rmses.loc['dropout_0.4', 'without_one_feature'] = np.min(abb_dp_valid_metric_4)
print(
    'Наименьшая RMSE сети с долей Dropout 0.6 и всеми признаками —',
    np.min(dp_valid_metric_6),
    f'— на {np.argmin(dp_valid_metric_6)*10} эпохе'
    )
rmses.loc['dropout_0.6', 'all_features'] = np.min(dp_valid_metric_6)
print(
    'Наименьшая RMSE сети с долей Dropout 0.6 и без одного признака —',
    np.min(abb_dp_valid_metric_6),
    f'— на {np.argmin(abb_dp_valid_metric_6)*10} эпохе',
    '\n'
    )
rmses.loc['dropout_0.6', 'without_one_feature'] = np.min(abb_dp_valid_metric_6)
print(
    'Наименьшая RMSE сети с долей Dropout 0.8 и всеми признаками —',
    np.min(dp_valid_metric_8),
    f'— на {np.argmin(dp_valid_metric_8)*10} эпохе'
    )
rmses.loc['dropout_0.8', 'all_features'] = np.min(dp_valid_metric_8)
print(
    'Наименьшая RMSE сети с долей Dropout 0.8 и без одного признака —',
    np.min(abb_dp_valid_metric_8),
    f'— на {np.argmin(abb_dp_valid_metric_8)*10} эпохе'
    )
rmses.loc['dropout_0.8', 'without_one_feature'] = np.min(abb_dp_valid_metric_8)

Наименьшая RMSE сети без Dropout и со всеми признаками — 4622.0547 — на 900 эпохе
Наименьшая RMSE сети без Dropout и одного признака — 4729.5415 — на 1340 эпохе 

Наименьшая RMSE сети с долей Dropout 0.2 и всеми признаками — 4569.503 — на 1130 эпохе
Наименьшая RMSE сети с долей Dropout 0.2 и без одного признака — 4565.6016 — на 2490 эпохе 

Наименьшая RMSE сети с долей Dropout 0.4 и всеми признаками — 4708.542 — на 920 эпохе
Наименьшая RMSE сети с долей Dropout 0.4 и без одного признака — 4551.4927 — на 3040 эпохе 

Наименьшая RMSE сети с долей Dropout 0.6 и всеми признаками — 4698.905 — на 610 эпохе
Наименьшая RMSE сети с долей Dropout 0.6 и без одного признака — 4739.0117 — на 1030 эпохе 

Наименьшая RMSE сети с долей Dropout 0.8 и всеми признаками — 4695.6313 — на 850 эпохе
Наименьшая RMSE сети с долей Dropout 0.8 и без одного признака — 4465.75 — на 2990 эпохе


In [37]:
# определение функции для обучения нейронной сети батчами
def batches_fit(
    net,
    optimizer,
    train_loss,
    valid_metric,
    patience=1,
    max_delta=0,
    batch_size=len(f_train),
    abb=0
    ):
    counter = 0
    min_valid_metric = np.inf
    num_batches = ceil(len(f_train)/batch_size)

    for epoch in range(num_epochs):
        order = np.random.permutation(len(f_train))
        epoch_loss_value = []

        for batch_idx in range(num_batches):
            start_index = batch_idx * batch_size
            optimizer.zero_grad()

            batch_indexes = order[start_index:start_index+batch_size]
            t_batch = t_train[batch_indexes]
            if abb == 1:
                f_batch = abb_f_train[batch_indexes]
            else:
                f_batch = f_train[batch_indexes]

            preds = net.forward(f_batch).flatten()

            loss_value = loss(preds, t_batch)
            epoch_loss_value.append(torch.sqrt(loss_value).detach())
            loss_value.backward()
    
            optimizer.step()

        if epoch % 10 == 0 or epoch == num_epochs - 1:
            train_loss.append(np.mean(epoch_loss_value))
            net.eval()
            if abb==1:
                valid_preds = net.forward(abb_f_valid).flatten()
            else:
                valid_preds = net.forward(f_valid).flatten()
            rmse = torch.sqrt(loss(
                valid_preds,
                t_valid
                )).detach()
            valid_metric.append(rmse)
            if rmse < min_valid_metric:
                min_valid_metric = rmse
                counter = 0
            elif rmse >= min_valid_metric:
                counter += 1
            if counter >= patience or rmse - min_valid_metric >= max_delta:
                break

In [38]:
# определение экземпляров сетей
bs_net_12 = NetELU(
    n_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons
    )
abb_bs_net_12 = NetELU(
    n_abb_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons
    )
bs_net_24 = NetELU(
    n_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons
    )
abb_bs_net_24 = NetELU(
    n_abb_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons
    )
bs_net_36 = NetELU(
    n_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons
    )
abb_bs_net_36 = NetELU(
    n_abb_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons
    )
bs_net_48 = NetELU(
    n_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons
    )
abb_bs_net_48 = NetELU(
    n_abb_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons
    )
bs_net_72 = NetELU(
    n_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons
    )
abb_bs_net_72 = NetELU(
    n_abb_in_neurons,
    n_hidden_neurons_1_m,
    n_hidden_neurons_2_m,
    n_out_neurons
    )
# определение оптимизаторов
bs_optimizer_12 = torch.optim.Adamax(bs_net_12.parameters(), lr=5e-2)
abb_bs_optimizer_12 = torch.optim.Adamax(abb_bs_net_12.parameters(), lr=5e-2)
bs_optimizer_24 = torch.optim.Adamax(bs_net_24.parameters(), lr=5e-2)
abb_bs_optimizer_24 = torch.optim.Adamax(abb_bs_net_24.parameters(), lr=5e-2)
bs_optimizer_36 = torch.optim.Adamax(bs_net_36.parameters(), lr=5e-2)
abb_bs_optimizer_36 = torch.optim.Adamax(abb_bs_net_36.parameters(), lr=5e-2)
bs_optimizer_48 = torch.optim.Adamax(bs_net_48.parameters(), lr=5e-2)
abb_bs_optimizer_48 = torch.optim.Adamax(abb_bs_net_48.parameters(), lr=5e-2)
bs_optimizer_72 = torch.optim.Adamax(bs_net_72.parameters(), lr=5e-2)
abb_bs_optimizer_72 = torch.optim.Adamax(abb_bs_net_72.parameters(), lr=5e-2)

# создание списков cо значениями метрик
bs_train_loss_12 = []
bs_valid_metric_12 = []
abb_bs_train_loss_12 = []
abb_bs_valid_metric_12 = []
bs_train_loss_24 = []
bs_valid_metric_24 = []
abb_bs_train_loss_24 = []
abb_bs_valid_metric_24 = []
bs_train_loss_36 = []
bs_valid_metric_36 = []
abb_bs_train_loss_36 = []
abb_bs_valid_metric_36 = []
bs_train_loss_48 = []
bs_valid_metric_48 = []
abb_bs_train_loss_48 = []
abb_bs_valid_metric_48 = []
bs_train_loss_72 = []
bs_valid_metric_72 = []
abb_bs_train_loss_72 = []
abb_bs_valid_metric_72 = []

In [39]:
# обучение нейронных сетей
batches_fit(
    bs_net_12,
    bs_optimizer_12,
    bs_train_loss_12,
    bs_valid_metric_12,
    20,
    100,
    12
    )
batches_fit(
    abb_bs_net_12,
    abb_bs_optimizer_12,
    abb_bs_train_loss_12,
    abb_bs_valid_metric_12,
    20,
    100,
    12,
    1
    )
batches_fit(
    bs_net_24,
    bs_optimizer_24,
    bs_train_loss_24,
    bs_valid_metric_24,
    20,
    100,
    24
    )
batches_fit(
    abb_bs_net_24,
    abb_bs_optimizer_24,
    abb_bs_train_loss_24,
    abb_bs_valid_metric_24,
    20,
    100,
    24,
    1
    )
batches_fit(
    bs_net_36,
    bs_optimizer_36,
    bs_train_loss_36,
    bs_valid_metric_36,
    20,
    100,
    36
    )
batches_fit(
    abb_bs_net_36,
    abb_bs_optimizer_36,
    abb_bs_train_loss_36,
    abb_bs_valid_metric_36,
    20,
    100,
    36,
    1
    )
batches_fit(
    bs_net_48,
    bs_optimizer_48,
    bs_train_loss_48,
    bs_valid_metric_48,
    20,
    100,
    48
    )
batches_fit(
    abb_bs_net_48,
    abb_bs_optimizer_48,
    abb_bs_train_loss_48,
    abb_bs_valid_metric_48,
    20,
    100,
    48,
    1
    )
batches_fit(
    bs_net_72,
    bs_optimizer_72,
    bs_train_loss_72,
    bs_valid_metric_72,
    20,
    100,
    72
    )
batches_fit(
    abb_bs_net_72,
    abb_bs_optimizer_72,
    abb_bs_train_loss_72,
    abb_bs_valid_metric_72,
    20,
    100,
    72,
    1
    )

In [40]:
# сравнение наилучших метрик сетей на валидации
print(
    'Наименьшая RMSE сети без батчей и со всеми признаками —',
    np.min(valid_metric_m),
    f'— на {np.argmin(valid_metric_m)*10} эпохе'
    )
print(
    'Наименьшая RMSE сети без батчей и одного признака —',
    np.min(abb_valid_metric_m),
    f'— на {np.argmin(abb_valid_metric_m)*10} эпохе',
    '\n'
    )
print(
    'Наименьшая RMSE сети с батчами размера 12 и всеми признаками —',
    np.min(bs_valid_metric_12),
    f'— на {np.argmin(bs_valid_metric_12)*10} эпохе'
    )
rmses.loc['n_batches_12', 'all_features'] = np.min(bs_valid_metric_12)
print(
    'Наименьшая RMSE сети с батчами размера 12 и без одного признака —',
    np.min(abb_bs_valid_metric_12),
    f'— на {np.argmin(abb_bs_valid_metric_12)*10} эпохе',
    '\n'
    )
rmses.loc['n_batches_12', 'without_one_feature'] = \
np.min(abb_bs_valid_metric_12)
print(
    'Наименьшая RMSE сети с батчами размера 24 и всеми признаками —',
    np.min(bs_valid_metric_24),
    f'— на {np.argmin(bs_valid_metric_24)*10} эпохе'
    )
rmses.loc['n_batches_24', 'all_features'] = np.min(bs_valid_metric_24)
print(
    'Наименьшая RMSE сети с батчами размера 24 и без одного признака —',
    np.min(abb_bs_valid_metric_24),
    f'— на {np.argmin(abb_bs_valid_metric_24)*10} эпохе',
    '\n'
    )
rmses.loc['n_batches_24', 'without_one_feature'] = \
np.min(abb_bs_valid_metric_24)
print(
    'Наименьшая RMSE сети с батчами размера 36 и всеми признаками —',
    np.min(bs_valid_metric_36),
    f'— на {np.argmin(bs_valid_metric_36)*10} эпохе'
    )
rmses.loc['n_batches_36', 'all_features'] = np.min(bs_valid_metric_36)
print(
    'Наименьшая RMSE сети с батчами размера 36 и без одного признака —',
    np.min(abb_bs_valid_metric_36),
    f'— на {np.argmin(abb_bs_valid_metric_36)*10} эпохе',
    '\n'
    )
rmses.loc['n_batches_36', 'without_one_feature'] = \
np.min(abb_bs_valid_metric_36)
print(
    'Наименьшая RMSE сети с батчами размера 48 и всеми признаками —',
    np.min(bs_valid_metric_48),
    f'— на {np.argmin(bs_valid_metric_48)*10} эпохе'
    )
rmses.loc['n_batches_48', 'all_features'] = np.min(bs_valid_metric_48)
print(
    'Наименьшая RMSE сети с батчами размера 48 и без одного признака —',
    np.min(abb_bs_valid_metric_48),
    f'— на {np.argmin(abb_bs_valid_metric_48)*10} эпохе',
    '\n'
    )
rmses.loc['n_batches_48', 'without_one_feature'] = \
np.min(abb_bs_valid_metric_48)
print(
    'Наименьшая RMSE сети с батчами размера 72 и всеми признаками —',
    np.min(bs_valid_metric_72),
    f'— на {np.argmin(bs_valid_metric_72)*10} эпохе'
    )
rmses.loc['n_batches_72', 'all_features'] = np.min(bs_valid_metric_72)
print(
    'Наименьшая RMSE сети с батчами размера 72 и без одного признака —',
    np.min(abb_bs_valid_metric_72),
    f'— на {np.argmin(abb_bs_valid_metric_72)*10} эпохе'
    )
rmses.loc['n_batches_72', 'without_one_feature'] = \
np.min(abb_bs_valid_metric_72)

Наименьшая RMSE сети без батчей и со всеми признаками — 4622.0547 — на 900 эпохе
Наименьшая RMSE сети без батчей и одного признака — 4729.5415 — на 1340 эпохе 

Наименьшая RMSE сети с батчами размера 12 и всеми признаками — 4623.172 — на 240 эпохе
Наименьшая RMSE сети с батчами размера 12 и без одного признака — 4708.7153 — на 320 эпохе 

Наименьшая RMSE сети с батчами размера 24 и всеми признаками — 4529.722 — на 550 эпохе
Наименьшая RMSE сети с батчами размера 24 и без одного признака — 4598.817 — на 970 эпохе 

Наименьшая RMSE сети с батчами размера 36 и всеми признаками — 4602.024 — на 540 эпохе
Наименьшая RMSE сети с батчами размера 36 и без одного признака — 4784.8135 — на 510 эпохе 

Наименьшая RMSE сети с батчами размера 48 и всеми признаками — 4522.1113 — на 1160 эпохе
Наименьшая RMSE сети с батчами размера 48 и без одного признака — 4718.141 — на 560 эпохе 

Наименьшая RMSE сети с батчами размера 72 и всеми признаками — 4600.0454 — на 1210 эпохе
Наименьшая RMSE сети с батчами

Окончательно оценим качество итоговой наилучшей сети — с использованием доли Dropout 0.8 — путём обработки тестовой выборки. Снова взглянем на соответствующий график "Факт — Прогноз".

In [41]:
# получение предсказаний
test_predictions = abb_dp_net_8.forward(abb_f_test).flatten().detach()

# построение графика
test_fig = go.Figure()
test_fig.add_trace(go.Bar(
    x=np.array(range(len(t_test))),
    y=t_test,
    name='Факт',
    width=.9,
    marker_color='dodgerblue'
    ))
test_fig.add_trace(go.Bar(
    x=np.array(range(len(test_predictions))),
    y=test_predictions,
    name='Прогноз',
    width=.3,
    marker_color='orange'
    ))
test_fig.update_layout(
    barmode='overlay',
    width=1000,
    height=500,
    legend_orientation='h',
    title=dict(
        text='Истинные и предсказанные температуры звёзд на тесте',
        x=.5
        ),
    xaxis_title='Номер звезды в выборке',
    yaxis_title='Температура звезды, К'
    )
test_fig.show()

# оценка RMSE на тестовой выборке
print(
    'RMSE итоговой нейросети: ',
    torch.sqrt(loss(test_predictions, t_test)).detach().numpy()
    )

RMSE итоговой нейросети:  2526.207


**Промежуточные выводы**

В итоге с помощью Dropout удалось ещё несколько улучшить значение метрики. Использование батчей при обучении в паре случаев также дало положительный результат по сравнению с бейзлайном, но меньше, чем применение Dropout.

Судя по графику соотношения значений таргета и предсказаний, получившаяся модель практически не ошибается на малых значениях таргета, основной вклад в итоговую ошибку вносят отдельные расхождения на примерах с высокой температурой, причём сеть склонна в первую очередь занижать предсказания относительно истинных значений.

Ниже можно увидеть сводные результаты обучения нейросети с помощью Dropout и батчей по сравнению с baseline-версией.

In [42]:
# получение результатов обучения
rmses.astype('float').style.background_gradient(cmap='Blues', axis=0)

,all_features,without_one_feature
baseline,4558.877930,4579.826172
dropout_0.2,4569.502930,4565.601562
dropout_0.4,4708.541992,4551.492676
dropout_0.6,4698.904785,4739.011719
dropout_0.8,4695.631348,4465.750000
n_batches_12,4623.171875,4708.715332
n_batches_24,4529.722168,4598.816895
n_batches_36,4602.023926,4784.813477
n_batches_48,4522.111328,4718.141113
n_batches_72,4600.045410,4694.480957


## Итоговый вывод

После нескольких последовательных улучшений путём перебора различных архитектур сетей и подходов к обучению была достигнута требуемая точность модели. На отложенной же тестовой выборке нейросеть и вовсе показала качество, на 44% лучше минимально допустимого — 2 526 К по сравнению с 4 500 К.

Наилучшее решение в итоге было основано на выборке со всеми предоставленными признаками, хотя в целом встречались и обратные результаты практически на каждом этапе сравнения различных архитектур/параметров сетей. Также можно сделать вывод, что для подобных достаточно малых по объёму данных достаточно решения с малым количеством слоёв для достижения приемлемого результата.

И обучение батчами, и Dropout в данном случае несколько улучшили метрику, также можно попробовать использовать BatchNorm вместо последнего.

Помимо наилучшей получившейся сети, показавшей неплохой результат как на валидационной выборке, так и на тестовой, можно обратить внимание и на модель, показавшую, наоборот, наихудшее качество при выбранном количестве эпох. На протяжении всех 10 000 итераций нейросеть с двумя скрытыми слоями, небольшим количеством нейронов и функцией активации ReLU, обученная на выборке со всеми признаками, очень медленно обучалась, причём судя по графику ниже, линейно, как на тренировке, так и на валидации, и достигла в итоге только 14 826 RMSE на последней эпохе. Вероятно, можно попробовать обучать её и дальше, используя сильно большее количество итераций; возможно, рано или поздно сеть достигнет лучшего качества, чем выбранная в итоге. Но вопрос в том, оправдано ли будет использование сильно большего количества ресурсов, временных и/или аппартаных, для достижения вероятно улучшенного результата.

In [43]:
# просмотр процесса тренировки наиболее медленно обучающейся сети
fit_plot(train_loss_l, valid_metric_l)